### Loading packages

In [55]:
import os
import sys
import io
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

import sqlite3
from sqlite3 import Error

#### Reading Data Folder

In [2]:
path = '../data'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
            files.append(os.path.join(r, file))

for f in files:
    print(f)

../data/RAW_recipes.csv
../data/interactions_test.csv
../data/test.json
../data/interactions_validation.csv
../data/PP_recipes.csv
../data/PP_users.csv
../data/ingr_map.pkl
../data/.~lock.RAW_interactions.csv#
../data/train.json
../data/311962_783630_bundle_archive.zip
../data/interactions_train.csv
../data/RAW_interactions.csv
../data/FAOSTAT_livestock_2018.csv
../data/FAOSTAT_crops_2018.csv


### [Food Data from Kaggle competition](https://www.kaggle.com/shuyangli94/food-com-recipes-and-user-interactions)

In [3]:
df_recipes_raw = pd.read_csv('../data/RAW_recipes.csv')
df_recipes_cod = pd.read_csv('../data/PP_recipes.csv')

df_recipes_raw.tags = df_recipes_raw.tags.apply(eval)
df_recipes_raw.ingredients = df_recipes_raw.ingredients.apply(eval)
df_recipes_raw.steps = df_recipes_raw.steps.apply(eval)

In [4]:
df_recipes_raw.head(3)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13


In [5]:
df_recipes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [6]:
df_recipes_cod.head(3)

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."


In [7]:
df_recipes_cod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178265 entries, 0 to 178264
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   id                 178265 non-null  int64 
 1   i                  178265 non-null  int64 
 2   name_tokens        178265 non-null  object
 3   ingredient_tokens  178265 non-null  object
 4   steps_tokens       178265 non-null  object
 5   techniques         178265 non-null  object
 6   calorie_level      178265 non-null  int64 
 7   ingredient_ids     178265 non-null  object
dtypes: int64(3), object(5)
memory usage: 10.9+ MB


In [8]:
df_recipes_raw[df_recipes_raw.id==424415]

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
8586,aromatic basmati rice rice cooker,424415,61,496803,2010-05-10,"[weeknight, time-to-make, course, main-ingredi...","[228.2, 2.0, 2.0, 8.0, 9.0, 1.0, 15.0]",6,"[rinse the rice in a fine strainer , then drai...",from the ultimate rice cooker cookbook. the a...,"[basmati rice, water, salt, cinnamon stick, gr...",5


In [9]:
df_recipes_cod[df_recipes_cod.id==424415]

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"


In [72]:
df_recipes = pd.merge(df_recipes_cod, df_recipes_raw, on='id')
df_recipes.set_index(["id"], drop=True, inplace=True)
df_recipes.drop(['contributor_id', 
                 'submitted', 
                 'n_steps', 
                 'description', 
                 'tags', 
                 'n_ingredients', 
                 'minutes',
                 'ingredient_tokens',
                 'steps_tokens',
                 'steps',
                 'techniques',
                 'calorie_level',
                 'nutrition',
                 'i'], axis=1, inplace=True)

df_recipes = df_recipes[['name',
                         'name_tokens',
                         'ingredients',
                         'ingredients_ids',

                         ]]
df_recipes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 178265 entries, 424415 to 263840
Data columns (total 4 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   name             178265 non-null  object
 1   name_tokens      178265 non-null  object
 2   ingredients      178265 non-null  object
 3   ingredients_ids  178265 non-null  object
dtypes: object(4)
memory usage: 6.8+ MB


In [73]:
df_recipes.head(10)

,name,name_tokens,ingredients,ingredients_ids
id,,,,
424415,aromatic basmati rice rice cooker,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[basmati rice, water, salt, cinnamon stick, gr...","[1392, 20, 6, 672, 2737]"
146223,pumpkin pie a la easy,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[flour, oats, brown sugar, pecans, butter, egg...","[106, 1003, 59, 242, 4, 9, 4092, 419, 241, 401..."
312329,cheesy tomato soup with potatoes,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[chicken broth, water, salt, black pepper, oni...","[565, 20, 6, 94, 257, 343, 93, 53, 184, 121, 6..."
74301,mini tacos,"[40480, 10025, 31156, 40481]","[wonton wrappers, hamburger, taco seasoning, s...","[2848, 1047, 424, 300, 12, 79, 2363, 2146]"
76272,rosemary s hanky panky s,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[ground beef, ground sausage, velveeta cheese,...","[13, 1231, 579, 5764]"
465171,pink bavarian crown strawberry dream supreme,"[40480, 3390, 829, 35873, 7047, 13731, 2640, 1...","[strawberry jell-o gelatin dessert, water, str...","[1291, 20, 662, 36, 501, 2292, 3863, 90, 4, 305]"
163861,tom and kelly s chicken fried steak,"[40480, 1966, 488, 5218, 252, 5867, 10994, 118...","[boneless pork chops, flour, salt, pepper, egg...","[648, 106, 6, 31, 144, 436, 8099, 107, 4]"
186383,chocolate oat cookie bars,"[40480, 5317, 7, 491, 11274, 5639, 40481]","[flour, oats, baking powder, baking soda, salt...","[106, 1003, 149, 77, 6, 59, 107, 150, 1107, 417]"
116395,tropical lemon cream bars,"[40480, 16190, 13249, 4914, 5639, 40481]","[flour, confectioners' sugar, powdered milk, c...","[106, 861, 1331, 2542, 4, 5924, 5974, 318, 144..."


#### Opening a dataframe for conversion of ingredients into simpler forms

In [58]:
with open('../data/ingr_map.pkl', 'rb') as f:
    ing = pickle.load(f)

In [83]:
ing.head()

,raw_ingr,raw_words,processed,len_proc,replaced,count,id
0,"medium heads bibb or red leaf lettuce, washed,...",13,"medium heads bibb or red leaf lettuce, washed,...",73,lettuce,4507,4308
1,mixed baby lettuces and spring greens,6,mixed baby lettuces and spring green,36,lettuce,4507,4308
2,romaine lettuce leaf,3,romaine lettuce leaf,20,lettuce,4507,4308
3,iceberg lettuce leaf,3,iceberg lettuce leaf,20,lettuce,4507,4308
4,red romaine lettuce,3,red romaine lettuce,19,lettuce,4507,4308


In [84]:
simple_ings = {}
for idx1 in ing.index:
    simple_ings[ing.loc[idx1, 'raw_ingr']] = ing.loc[idx1, 'replaced'] 
len(simple_ings)

11659

#### Creating a dictionaire for ingredients

In [87]:
ingredients = {}

for idx1 in df_recipes.index:
        for idx2 in range(len(df_recipes.loc[idx1, 'ingredients'])):
            try:
                ingredients[df_recipes.loc[idx1, 'ingredients_ids'][idx2]] = simple_ings[df_recipes.loc[idx1, 'ingredients'][idx2]]
            except:
                ingredients[df_recipes.loc[idx1, 'ingredients_ids'][idx2]] = df_recipes.loc[idx1, 'ingredients'][idx2]
                #print(df_recipes.loc[idx1, 'ingredients'][idx2])
len(ingredients)

13627

#### Creating the first graph of recipes and ingredients

https://developer.edamam.com/edamam-docs-recipe-api